In [1]:
import numpy as np
import numpy.ma as ma
from numpy import arange,array,ones,linalg

from mpl_toolkits import mplot3d

import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = False
from matplotlib import cm, colors
from matplotlib.patches import Ellipse
import matplotlib as mpl

from scipy.optimize import minimize
from scipy import ndimage
from scipy.ndimage import zoom, gaussian_filter1d
from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization.wcsaxes import SphericalCircle
from astropy.nddata import bitmask
from astropy.nddata.bitmask import BitFlagNameMap

import math

from tqdm import tqdm

import copy

import os

from sympy.interactive import printing
printing.init_printing(use_latex = True)

import warnings

# For memory debugging
from collections import Counter
import linecache
import tracemalloc

In [2]:
current_directory = os.getcwd()
print(current_directory)

/global/u1/k/kadglass/DESI_SGA/Morphology


In [3]:
#SGA_Param_Data = Table.read('SGA_Asym_Data.fits')
# SGA_Param_Data = Table.read('SGA_M20_Data1.fits')
SGA_Param_Data = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA_Param_Data[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [4]:
# Bits in the dict
MASKBITS = dict(
    NPRIMARY   = 0x1,   # not PRIMARY
    BRIGHT     = 0x2,
    SATUR_G    = 0x4,
    SATUR_R    = 0x8,
    SATUR_Z    = 0x10,
    ALLMASK_G  = 0x20,
    ALLMASK_R  = 0x40,
    ALLMASK_Z  = 0x80,
    WISEM1     = 0x100, # WISE masked
    WISEM2     = 0x200,
    BAILOUT    = 0x400, # bailed out of processing
    MEDIUM     = 0x800, # medium-bright star
    GALAXY     = 0x1000, # SGA large galaxy
    CLUSTER    = 0x2000, # Cluster catalog source
    SATUR_I    = 0x4000,
    ALLMASK_I  = 0x8000,
    SUB_BLOB   = 0x10000,
)

In [5]:
def makecompactimage(mmc_image, mask_c):
    # extract the image shape and coordinates
    rows, cols = mmc_image.shape
    y, x = np.mgrid[0:rows, 0:cols]
    squared_distance = x**2 + y**2
    
    # flatten the images
    flattened = np.ravel(mmc_image)
    
    # sort the indices by assosiated intensity pixel, greatest to lowest intensity
    sorted_pix = np.argsort(flattened)[::-1]

    # make a list of intensity values, replacing masked values with 0
    filledflattened = flattened.filled(0.0)[sorted_pix]

    # sort the indices replaced with zeros masked intensity, greatest to lowest
    sorted_filled_pix = np.argsort(filledflattened)[::-1]

    # sort the intensities of the replaced masked values
    filledintlist = filledflattened[sorted_filled_pix]
    
    # find the index of the brightest pixel in the original image
    brightest_pixel_index = np.unravel_index(np.argmax(mmc_image, axis=None), mmc_image.shape)

    # calculate squared distances from the center (brightest pixel)
    squared_distances_centered = (x - brightest_pixel_index[1])**2 + (y - brightest_pixel_index[0])**2

    # sort the pixels based on squared distances from the center
    sorted_distances_centered = np.argsort(squared_distances_centered.flatten())

    # create a circular arrangement around the brightest pixel
    arranged_image = np.zeros_like(mmc_image)
    arranged_image.flat[sorted_distances_centered] = filledintlist

    compact_image = ma.array(arranged_image, mask=mask_c)
    
    return compact_image

In [6]:
def potentialenergycalc(mmc_image):
    '''Note: this function is using too much memory and needs to be rewritten.'''
    
    # get pixel coordinates for non-masked pixels of image
    y, x = np.where(~mmc_image.mask)
    
    # rij_matrix where each element represents the distance between the corresponding pixel pairs
    # distance = sqrt{ (x_2 - x_1)^2 + (y_2 - y_1)^2
    rij_matrix = np.sqrt((x[:, np.newaxis] - x)**2 + (y[:, np.newaxis] - y)**2)
    print('*** rij_matrix:', rij_matrix.shape)
    flux_matrix = mmc_image[y, x]
    print('*** flux_matrix:', flux_matrix.shape)
    
    # apply the mask and sum the potential energy
    potential_energy = 0.5*(np.sum(flux_matrix[np.newaxis, :] * flux_matrix[:, np.newaxis] / rij_matrix))
    return potential_energy

In [7]:
def multcalc(compact_potetnial_energy, actual_potential_energy):
    multiplicity = 100*(np.log10(compact_potetnial_energy / actual_potential_energy))
    return multiplicity

In [8]:
# For memory tracing
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, '<frozen importlib.bootstrap>'), 
        tracemalloc.Filter(False, '<unknown>'),
    ))
    top_stats = snapshot.statistics(key_type)
    
    print('Top %s lines' % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print('#%s: %s:%s: %.1f KiB' % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)
    
    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print('%s other: %.1f KiB' % (len(other), size/1024))
    
    total = sum(stat.size for stat in top_stats)
    print('Total allocated size: %.1f KiB' % (total/1024))

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

# For memory tracing
tracemalloc.start()

# Create a list to store indices of processed galaxies
processed_galaxies = []
no_bitmask = []
no_image_path = []
image_shape_unknown = []

# For memory tracing
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')
print('Created lists: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

n_galaxies = len(SGA_Param_Data)

# For memory tracing
snapshot2 = tracemalloc.take_snapshot()
top_stats = snapshot2.compare_to(snapshot, 'lineno')
print('Calculate n_galaxies: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

SGA_Param_Data['Mult_g'] = np.NaN * np.ones(n_galaxies)
SGA_Param_Data['Mult_r'] = np.NaN * np.ones(n_galaxies)
SGA_Param_Data['Mult_z'] = np.NaN * np.ones(n_galaxies)

# For memory tracing
snapshot3 = tracemalloc.take_snapshot()
top_stats = snapshot3.compare_to(snapshot2, 'lineno')
print('Initialize Mult_ columns: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

galaxy_range = range(85, 88)

for i in tqdm(galaxy_range, desc="Processing Galaxies", unit="galaxy", leave=True):
    f1 = math.trunc(SGA_Param_Data['RA'][i])
    f2 = math.trunc((1000*SGA_Param_Data['RA'][i]))
    f3 = math.trunc((100000*SGA_Param_Data['RA'][i]))
    f4 = 'm' if SGA_Param_Data['DEC'][i] < 0 else 'p'
    f5 = abs(math.trunc((100000*SGA_Param_Data['DEC'][i])))
    path = '/pscratch/sd/b/barnerg/SGA2024/cutouts/{:03d}/{:06d}/{:08d}{}{:07d}.fits'.format(f1,f2,f3,f4,f5)
    
    # For memory tracing
    snapshot4 = tracemalloc.take_snapshot()
    top_stats = snapshot4.compare_to(snapshot3, 'lineno')
    print('Defined file path: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
        
    if os.path.exists(path):
        fits_file = fits.open(path, ignore_missing_simple=True, ignore_missing_end=True)
        
        # For memory tracing
        snapshot5 = tracemalloc.take_snapshot()
        top_stats = snapshot5.compare_to(snapshot4, 'lineno')
        print('Open fits file: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

        try:
            fits_file[1].data
        except:
            no_bitmask.append(path)
            var_exists = False
            fits_file.close()
        else:
            var_exists = True
            
        # For memory tracing
        snapshot6 = tracemalloc.take_snapshot()
        top_stats = snapshot6.compare_to(snapshot5, 'lineno')
        print('Try-except to check bitmask: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

        if var_exists:
            image = fits_file[0].data
            bit_mask = fits_file[1].data
            fits_file.close()
            
            print('*** Image:', image.shape, image.dtype)
            print('*** Bitmask:', bit_mask.shape, bit_mask.dtype)
            
            # For memory tracing
            snapshot7 = tracemalloc.take_snapshot()
            top_stats = snapshot7.compare_to(snapshot6, 'lineno')
            print('Store image & bitmask: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
            
            # reshape the data
            # checking to see if the galaxy has 3 color channels
            if image.shape[0] == 3:
                # the tuple swaps the 1st and 3rd dimensions, resulting in a new array, and assigns this array to the variable image_data
                image_data = np.transpose(image, (1, 2, 0))
                #print("image_data shape:", image_data.shape)
                
                # For memory tracing
                snapshot8 = tracemalloc.take_snapshot()
                top_stats = snapshot8.compare_to(snapshot7, 'lineno')
                print('Transpose image: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                mg_image = ma.array(image_data[:,:,0])
                mr_image = ma.array(image_data[:,:,1])
                mz_image = ma.array(image_data[:,:,2])
                
                # For memory tracing
                snapshot9 = tracemalloc.take_snapshot()
                top_stats = snapshot9.compare_to(snapshot8, 'lineno')
                print('Create 3 masked image arrays: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                # Generate bitmask to apply to image
                condition = bit_mask & MASKBITS['GALAXY'] != 0
                mask_ = condition.astype(int)
                
                # For memory tracing
                snapshot10 = tracemalloc.take_snapshot()
                top_stats = snapshot10.compare_to(snapshot9, 'lineno')
                print('Generate bitmask: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

                # Generate bitmask to apply to image
                condition2 = mask_ & (MASKBITS['BRIGHT'] | MASKBITS['SATUR_G'] | MASKBITS['SATUR_R'] | MASKBITS['SATUR_Z'] | MASKBITS['WISEM1'] | MASKBITS['WISEM2']) != 0
                mask_2 = mask_ & ~condition2
                
                # For memory tracing
                snapshot11 = tracemalloc.take_snapshot()
                top_stats = snapshot11.compare_to(snapshot10, 'lineno')
                print('Create bitmask2: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

                # make the mask (318, 318, 3), where the third dimension represents the same 2D mask array duplicated three times
                mask = np.dstack([mask_2.astype(bool), mask_2.astype(bool), mask_2.astype(bool)])
                mimage_data = ma.array(image_data, mask=~mask)
                
                # For memory tracing
                snapshot12 = tracemalloc.take_snapshot()
                top_stats = snapshot12.compare_to(snapshot11, 'lineno')
                print('Create final mask & masked image: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

                # masked color images
                mmg_image = ma.array(image_data[:,:,0], mask=~mask_2.astype(bool))
                mmr_image = ma.array(image_data[:,:,1], mask=~mask_2.astype(bool))
                mmz_image = ma.array(image_data[:,:,2], mask=~mask_2.astype(bool))
                
                # For memory tracing
                snapshot13 = tracemalloc.take_snapshot()
                top_stats = snapshot13.compare_to(snapshot12, 'lineno')
                print('Mask color arrays: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

                # Use the masks from the original images
                mask_g = mmg_image.mask
                mask_r = mmr_image.mask
                mask_z = mmz_image.mask
                
                # For memory tracing
                snapshot14 = tracemalloc.take_snapshot()
                top_stats = snapshot14.compare_to(snapshot13, 'lineno')
                print('Create copies of the color masks: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                # call makecompactimage for each channel
                compactimageg = makecompactimage(mmg_image, mask_g)
                compactimager = makecompactimage(mmr_image, mask_r)
                compactimagez = makecompactimage(mmz_image, mask_z)
                
                # For memory tracing
                snapshot15 = tracemalloc.take_snapshot()
                top_stats = snapshot15.compare_to(snapshot14, 'lineno')
                print('Rearrange pixels via makecompactimage(): %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                # call potentialenergycalc for each channel to calculate actual and compact potential
                actualpotentialenergyg = potentialenergycalc(mmg_image)
                actualpotentialenergyr = potentialenergycalc(mmr_image)
                actualpotentialenergyz = potentialenergycalc(mmz_image)
                
                # For memory tracing
                snapshot16 = tracemalloc.take_snapshot()
                top_stats = snapshot16.compare_to(snapshot15, 'lineno')
                print('Calculate potential energy of original images: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                compactpotentialenergyg = potentialenergycalc(compactimageg)
                compactpotentialenergyr = potentialenergycalc(compactimager)
                compactpotentialenergyz = potentialenergycalc(compactimagez)
                
                # For memory tracing
                snapshot17 = tracemalloc.take_snapshot()
                top_stats = snapshot17.compare_to(snapshot16, 'lineno')
                print('Calculate potential energy of sorted images: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                # call multcalc
                multiplicityg = multcalc(compactpotentialenergyg, actualpotentialenergyg)
                multiplicityr = multcalc(compactpotentialenergyr, actualpotentialenergyr)
                multiplicityz = multcalc(compactpotentialenergyz, actualpotentialenergyz)
                
                # For memory tracing
                snapshot18 = tracemalloc.take_snapshot()
                top_stats = snapshot18.compare_to(snapshot17, 'lineno')
                print('Calculate multiplicity for each color: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))
                
                # append values
                SGA_Param_Data[i]['Mult_g'] = multiplicityg
                SGA_Param_Data[i]['Mult_r'] = multiplicityr
                SGA_Param_Data[i]['Mult_z'] = multiplicityz
                
                # For memory tracing
                snapshot19 = tracemalloc.take_snapshot()
                top_stats = snapshot19.compare_to(snapshot18, 'lineno')
                print('Add multiplicity to columns: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

                # Append the index of the processed galaxy to the list
                processed_galaxies.append(i)
                print("Galaxy processed!", i, path)
                
                # For memory tracing
                snapshot20 = tracemalloc.take_snapshot()
                top_stats = snapshot20.compare_to(snapshot19, 'lineno')
                print('Append processed index to list: %.1f KiB' % (sum(stat.size for stat in top_stats)/1024))

            else:
                image_shape_unknown.append(path)
    else: 
        no_image_path.append(path)

# For memory tracing
snapshot = tracemalloc.take_snapshot()
display_top(snapshot, limit=10)

Created lists: 1.6 KiB
Calculate n_galaxies: 8.0 KiB
Initialize Mult_ columns: 9014.1 KiB


Processing Galaxies:   0%|          | 0/3 [00:00<?, ?galaxy/s]

Defined file path: 9165.1 KiB
Open fits file: 13365.8 KiB
Try-except to check bitmask: 13424.1 KiB
***Image: (3, 213, 213) >f4
***Bitmask: (213, 213) >i4
Store image & bitmask: 9346.7 KiB
Transpose image: 9359.7 KiB
Create 3 masked image arrays: 9380.9 KiB
Generate bitmask: 9800.6 KiB
Create bitmask2: 10227.3 KiB
Create final mask & masked image: 10521.2 KiB
Mask color arrays: 10678.1 KiB
Create copies of the color masks: 10697.9 KiB
Rearrange pixels via makecompactimage(): 11543.0 KiB
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)
Calculate potential energy of original images: 11591.3 KiB
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)
*** rij_matrix: (4015, 4015)
*** flux_matrix: (4015,)


Processing Galaxies:  33%|███▎      | 1/3 [00:04<00:09,  4.98s/galaxy]

Calculate potential energy of sorted images: 11634.8 KiB
Calculate multiplicity for each color: 11660.2 KiB
Add multiplicity to columns: 11688.8 KiB
Galaxy processed! 85 /pscratch/sd/b/barnerg/SGA2024/cutouts/190/190297/19029721p6607370.fits
Append processed index to list: 11716.6 KiB
Defined file path: 11746.9 KiB
Open fits file: 15857.8 KiB
Try-except to check bitmask: 15892.8 KiB
***Image: (3, 253, 253) >f4
***Bitmask: (253, 253) >i4
Store image & bitmask: 11790.8 KiB
Transpose image: 11796.4 KiB
Create 3 masked image arrays: 11801.0 KiB
Generate bitmask: 11971.2 KiB
Create bitmask2: 12140.4 KiB
Create final mask & masked image: 12254.7 KiB
Mask color arrays: 12451.1 KiB
Create copies of the color masks: 12323.6 KiB
Rearrange pixels via makecompactimage(): 12656.7 KiB
*** rij_matrix: (12831, 12831)
*** flux_matrix: (12831,)
*** rij_matrix: (12831, 12831)
*** flux_matrix: (12831,)
*** rij_matrix: (12831, 12831)
*** flux_matrix: (12831,)
Calculate potential energy of original images: 

Processing Galaxies:  67%|██████▋   | 2/3 [00:49<00:28, 28.09s/galaxy]

Calculate potential energy of sorted images: 12646.8 KiB
Calculate multiplicity for each color: 12645.7 KiB
Add multiplicity to columns: 12643.9 KiB
Galaxy processed! 86 /pscratch/sd/b/barnerg/SGA2024/cutouts/120/120157/12015780p3717878.fits
Append processed index to list: 12644.1 KiB
Defined file path: 12647.9 KiB
Open fits file: 16758.8 KiB
Try-except to check bitmask: 16780.2 KiB
***Image: (3, 445, 445) >f4
***Bitmask: (445, 445) >i4
Store image & bitmask: 12674.9 KiB
Transpose image: 12674.8 KiB
Create 3 masked image arrays: 12672.8 KiB
Generate bitmask: 13850.6 KiB
Create bitmask2: 15030.6 KiB
Create final mask & masked image: 15816.7 KiB
Mask color arrays: 16398.5 KiB
Create copies of the color masks: 16211.6 KiB
Rearrange pixels via makecompactimage(): 18573.4 KiB


In [16]:
print(f"Galaxies with no bitmask: {len(no_bitmask)}")
print(f"Galaxies with no image path: {len(no_image_path)}")
print(f"Galaxies with an unknown image shape: {len(image_shape_unknown)}")
print(f"Galaxies successfully processed: {len(processed_galaxies)}")
print(f"All Galaxies Processed: {len(no_bitmask) + len(no_image_path) + len(image_shape_unknown) + len(processed_galaxies)}")
print(f"Total Galaxies: {len(SGA_Param_Data)}")

Galaxies with no bitmask: 0
Galaxies with no image path: 0
Galaxies with an unknown image shape: 0
Galaxies successfully processed: 5
All Galaxies Processed: 5
Total Galaxies: 383620


In [17]:
print(SGA_Param_Data)

 SGA_ID    SGA_GALAXY    ...       Mult_r             Mult_z      
------- ---------------- ... ------------------ ------------------
      2       SGA-2020 2 ... 1.1263294773235446 1.0283820455741772
      3       SGA-2020 3 ...  4.086030804117246  4.462755910463722
      4       SGA-2020 4 ... 0.8782154875988185 1.3895103552640973
      7       SGA-2020 7 ... 3.2538569008947174 3.5529376711243503
     18      SGA-2020 18 ... 0.4696566732261533 1.2766460829880248
     20      SGA-2020 20 ...                nan                nan
     24      SGA-2020 24 ...                nan                nan
     25      SGA-2020 25 ...                nan                nan
     26      SGA-2020 26 ...                nan                nan
     29      SGA-2020 29 ...                nan                nan
    ...              ... ...                ...                ...
5005190 SGA-2020 5005190 ...                nan                nan
5005195 SGA-2020 5005195 ...                nan               